### Vector-space Retrieval & Evaluation

### (1) Step 1: Load Inverted Index (H) and compute DocLen (DL).

In [1]:
import csv
import math

tindexfile = 'medline_term_index.csv'
invindexfile = 'medline_inverted_index.csv'
dindexfile = 'medline_doc_index.csv'

# Number of documents in the corpus (hard-coded for this corpus)
N = 1033

# Major data structures
H_invindex = {} # inverted index; term -> (idf, L:hashmap of (docID . tf))
DL_doclen = {}  # document lengths; docID -> len

## (1) Read the term index file and populate the invindex first
tid2term_map = {} # temporary storage to hold mappings of termID -> term

fin = open(tindexfile, 'r', encoding='utf-8')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    term = line[0]    # term string
    termID = line[1]  # termID
    df = int(line[2]) # document frequency
    idf = math.log10(N/df) # idf
    # record term -> (idf, emptyL) in H
    H_invindex[term] = (idf, dict())
    # record termID -> term 
    tid2term_map[termID] = term 
fin.close()

## (2) Read the inverted index file and add postings lists in H.
## Also compute document lengths too, incrementally -- and record in DL.
fin = open(invindexfile, 'r')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    termID = line[0]
    idx = 1
    while idx < (len(line)-1):
        docID = line[idx]
        tf = int(line[idx+1]) # raw tf of the term in this document
        # Record docID -> tf in term's L
        L = (H_invindex[tid2term_map[termID]])[1]
        L[docID] = tf  # docID -> raw term frequency
        
        # Accumulate the component vector length for the document
        tfidf = tf * (H_invindex[tid2term_map[termID]])[0] # tf * idf
        tfidfsq = math.pow(tfidf, 2.0)
        if docID in DL_doclen:
            DL_doclen[docID] += tfidfsq
        else:
            DL_doclen[docID] = tfidfsq
        #
        idx += 2
fin.close()

# Fix the DL entries by applying sqrt to make vector length.
for docID in DL_doclen.keys():
    val = DL_doclen[docID]
    DL_doclen[docID] = math.sqrt(val)

    
print ('Total # terms: %d' % len(H_invindex))
for term in ['pentobarbit', 'defici', 'treatment']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(H_invindex[term][1]), H_invindex[term][0]))

print ('\nTotal # documents: %d' % len(DL_doclen))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, DL_doclen[docID]))


Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
 - Entry for 'defici': df=39, idf=1.4230357144931214
 - Entry for 'treatment': df=172, idf=0.7785718746120717

Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


### (2) Step 2: Queries as Vectors

In [2]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

queryfile = 'medline.query'

# A list of queries. Each Query is a tuple, (qID, Q:term->tf map)
Queries_list = []

fin = open(queryfile, 'r', encoding='utf-8')#'iso-8859-1')
porter = nltk.PorterStemmer()

for line in fin:
    matchObj = re.match(r'^(\d+)\s+(.*)', line)
    if not matchObj:
        print ("ERROR with line -- %s" % line)
    else:
        queryID = matchObj.group(1) # queryID
        text = matchObj.group(2)    # query string (ignoring sentences)

        # process text string -- same processing as one applied to documents.
        tokens = word_tokenize(text.lower())
        terms = [porter.stem(w) for w in tokens if w not in stopwords.words('english') and len(w) > 1] # (**)
        # term frequencies of the terms in this query are obtained by NLTK's FreqDist
        fdist = nltk.FreqDist(terms)
        # append the Query in the list
        Queries_list.append((queryID, dict(fdist)))
fin.close()

print ('Total # queries: %d' % len(Queries_list))
for qid in [1, 21]:
    print (' - Query %s: %s' % (Queries_list[qid][0], Queries_list[qid][1]))

Total # queries: 30
 - Query 2: {'relationship': 1, 'blood': 1, 'cerebrospin': 1, 'fluid': 1, 'oxygen': 1, 'concentr': 1, 'partial': 1, 'pressur': 1, 'method': 1, 'interest': 1, 'polarographi': 1}
 - Query 22: {'mycoplasma': 1, 'infect': 1, 'presenc': 1, 'embryo': 1, 'fetu': 1, 'newborn': 1, 'infant': 1, 'anim': 1, 'pregnanc': 1, 'gynecolog': 1, 'diseas': 1, 'relat': 1, 'chromosom': 2, 'abnorm': 1}


### (3) Retrieval and Ranking 

### - For each query, apply the Vector Space Retrieval Algorithm and obtain a ranked list of retrieved documents (sorted by the cosine measure) 

In [318]:
import numpy as np

In [319]:
qlst=[]

for c in range(len(Queries_list)):
    hash_map_score={}
    q_length=[]
    
    for i in Queries_list[c][1]:
        
        doc_collect=[]
        try:
            idf=H_invindex[i][0]
        except:
            continue
        for d,f in H_invindex[i][1].items():
            doc_collect.append((d,f*idf*Queries_list[c][1][i]*idf)) #tfidf Dt *tfidf Qt

        for d_score in doc_collect:
            if d_score[0] in hash_map_score.keys():
                hash_map_score[d_score[0]]+=d_score[1]
            else:
                hash_map_score[d_score[0]]=d_score[1]
        
        q_length.append(Queries_list[c][1][i]*idf)
        
    q_length1=np.sqrt(np.dot(q_length,q_length))
    qlst.append([Queries_list[c][0],hash_map_score,q_length1])
        
    

In [320]:
cosine_score={}
for i in range(len(qlst)):
    d_collection=[]
    for (k,v) in qlst[i][1].items():
        d_collection.append((k,v/(DL_doclen[k]*qlst[i][2])))
        
    tb=sorted(d_collection, key=lambda tup: tup[1],reverse=True)
    cosine_score[qlst[i][0]]=tb
    

### (4) Evaluation -- Compute MAP
### - Compare the ranked lists with the anwers, and compute the MAP score.

For each query:
1.	I first sort the relevant documents based on the document ranking order from the cosine score computed in the previous part (Part 3 Retrieval and Ranking)
2.	then I compute the precision score for the relevant document and append it to an empty list
3.	lastly, in the end of the loop I will compute the average precision score by summing up all the value in the list where I stored the precision values for the relevant document and divided it by the length of that list.
4.	Then I append the average precision score for each query to another list that is used to store the average precision score for each list.
5.After each query is loop through, to compute the mean average precision score, I summed up all the average precision score for each query and divided it by the length of that list.



In [323]:
# import file
fin = open('medline.rel', 'r', encoding='utf-8')
ls=[]
for line in fin:
    ls.append(re.split("\n",line))
    
# relevancy answers formating
lst={}
for i in range(len(ls)):
    lst[ls[i][0].split()[0]]=ls[i][0].split()[1:]

In [324]:
query_lst=[]

for i in cosine_score.keys():
    doc_count={}
    for d_key in return_key(cosine_score[i]):
        doc_count[d_key]=0
        
    for relavant in lst[i]:
        if relavant in doc_count.keys():
            doc_count[relavant]+=1  
    
    query_lst.append((i,[c for c in enumerate(doc_count.items(),1)]))

In [325]:
avg_precision=[]

for i in range(len(query_lst)):
    numerator=0
    precision_collect=[]
    
    for a in range(len(query_lst[i][1])):
        if query_lst[i][1][a][1][1]!= 0:
            numerator+=1
            precision_collect.append(numerator/query_lst[i][1][a][0])
            
    avg_precision.append(sum(precision_collect)/len(precision_collect))

In [328]:
# the final MAP 
sum(avg_precision)/len(avg_precision)

0.5656605218653455